In [1]:
import os
import sys
import pandas as pd
import numpy as np
module_path = os.path.abspath(os.path.join('../../data/..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%matplotlib inline

In [2]:
businessPrepped = pd.read_pickle('../../data/prep/businessPrepped.pkl')

In [3]:
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0


In [4]:

#Returns NaN for closed businesses, and the current star rating for open businesses.
def openStars(row):
    if row['is_open'] == 0:
       return np.nan
    return row['stars']

def closedStars(row):
    if row['is_open'] == 1:
       return np.nan
    return row['stars']


In [5]:
#businessPrepped['stars'] = businessPrepped['stars'].astype(float)
businessPrepped['open_stars'] = businessPrepped.apply(lambda row : openStars(row), axis=1)
businessPrepped['closed_stars'] = businessPrepped.apply(lambda row : closedStars(row), axis=1)
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN


In [6]:

#Returns "" for closed businesses, and the current current category for open businesses.
def openCategories(row):
    if row['is_open'] == 0:
       return ""
    return row['categories']

def closedCategories(row):
    if row['is_open'] == 1:
       return ""
    return row['categories']



In [7]:
businessPrepped['closed_categories'] = businessPrepped.apply(lambda row : closedCategories(row), axis=1)
businessPrepped['open_categories'] = businessPrepped.apply(lambda row : openCategories(row), axis=1)
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars,closed_categories,open_categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN,,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN,,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN,,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0,Sporting Goods;Shopping,
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN,,American (New);Nightlife;Bars;Sandwiches;Ameri...


Let's try create area profiles; #businesses, #open, #reviews, avg. stars, avg. starsOpen,category dictionary

starsOpen : Average Stars of Open Businesses in an area.

In [8]:
def categoryAgg(categoryColumn):
    areaCategories= {}
    for value in categoryColumn:
        categoryList = value.lower().split(";")
        for category in categoryList:
            if category in areaCategories:
                areaCategories[category] += 1
            elif category != "":
                areaCategories[category] = 1
    return areaCategories

In [9]:
#Operates on 'name' column and returns a dictionary counting the number of different chains.
def chainAgg(chainColumn):
    #Self-determined list of chains based on their appearances in dataset.
    chainList = ['"Starbucks"', '"McDonald"\'s','"Subway"', '"Pizza Hut"','"Taco Bell"','"Burger King"','"Walgreens"','"Wendy\'s"',\
                '"The UPS Store"', '"Tim Hortons"']
    areaChains= {}
    for chain in chainColumn:
        if chain in chainList:
            if chain in areaChains:
                areaChains[chain] += 1
            else :
                areaChains[chain] = 1
    return areaChains

In [15]:
#make second stars column for std. deviation of ratings
businessPrepped['std.dev_stars'] = businessPrepped['stars']

zipProfileGroups = businessPrepped.groupby('postal_code').agg({'business_id':'count', 'is_open':sum, 'review_count':'mean',\
                                                               'checkins':'mean', 'tipcount':'mean',\
                                                               'open_stars':'mean', 'closed_stars':'mean',\
                                                              'std.dev_stars':"std" , 'open_categories':categoryAgg, \
                                              'closed_categories':categoryAgg, 'name':chainAgg})

In [16]:
zipProfileDf = pd.DataFrame(zipProfileGroups)

In [18]:
zipProfileDf['%closed'] = (zipProfileDf['business_id'] - zipProfileDf['is_open']) / zipProfileDf['business_id']

zipProfileDf[zipProfileDf['business_id'] > 20].sort_values('%closed', ascending = False).head(1)

,business_id,is_open,review_count,checkins,tipcount,open_stars,closed_stars,std.dev_stars,open_categories,closed_categories,name,%closed
postal_code,,,,,,,,,,,,
15289,23,7,36.173913,90.652174,5.956522,3.642857,3.5,0.76742,"{'museums': 1, 'colleges & universities': 2, '...","{'sandwiches': 1, 'restaurants': 13, 'bakeries...",{},0.695652


We will now alter the column titles to accurately reflect their contents. We will also rearrange the columns as desired.

In [37]:
zipProfileDf.columns=['#businesses','#open','#reviews','#checkins','#tips','open_stars','closed_stars','std.dev_stars',\
                      'open_categories','closed_categories','chains','%closed']

#Now Rearrange Columns
zipProfileDf = zipProfileDf[['#businesses','#open','%closed','#reviews','#checkins','#tips','open_stars','closed_stars',\
                             'std.dev_stars','open_categories','closed_categories','chains']]

zipProfileDf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1461 entries, 02224 to nan
Data columns (total 12 columns):
#businesses          1461 non-null int64
#open                1461 non-null int64
%closed              1461 non-null object
#reviews             1461 non-null float64
#checkins            1461 non-null float64
#tips                1461 non-null float64
open_stars           1461 non-null float64
closed_stars         1426 non-null float64
std.dev_stars        919 non-null float64
open_categories      1166 non-null float64
closed_categories    1461 non-null object
chains               1461 non-null object
dtypes: float64(7), int64(2), object(3)
memory usage: 148.4+ KB


In [38]:
zipProfileDf.to_pickle('../../data/analysis/ZIPprofiles.pkl')